In [2]:
import pandas as pd 
import numpy as np
import tqdm
import copy
import random
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os
import glob
import re

import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
#SPECIES 1 can be changed to any other species/class
#This script looks for the best n_frames and threshold factor by species! Find the best performing parameters for each species.

thresholds = range(0,3)

n_frames = range(0,30)  # Number of frames to wait after going below the threshold

#Change the pattern if needed
file_path = './04_Datasets/dataset_1/boundingboxes_class1.csv' #This example does species 1
#file_path = glob.glob(file_pattern)

for threshold in thresholds:
    for nFrames in n_frames:
        currentCSV = pd.read_csv(file_path)
        currentCSV_transformed = currentCSV.T
        
        #species = currentCSV_transformed.iloc[6, :]
        sequences = currentCSV_transformed.iloc[3:, :]
        
        sequences.fillna(0, inplace=True) #Padd with 0s
        
        # List to store sums for each column
        sums_dict = {}

        # Loop through each column (excluding timestamp)
        for col in sequences.columns[0:]:
            # Loop through each row
            sums_list = []
            above_threshold = False  # Flag to track if the value is above threshold
            counter = 0  # Counter to track the number of frames after going below threshold
            counterFP = 0 # Initialise the false-positive counter that will not allow a single frame detection
            
            for i,(index,row) in enumerate(sequences.iterrows()):
                
                # Check if the value is above the threshold
                if row[col] > threshold and not above_threshold:
                    above_threshold = True
                    start_index = index  # Store the start index for this interval
                    counter = 0  # Reset the counter when going above threshold
                    counterFP = counterFP + 1
        
                # Check if the value stays above the threshold
                elif row[col] > threshold and above_threshold:
                    counterFP = counterFP + 1
                    
                # Check if the value drops below the threshold again but give a buffer of nFrames
                elif row[col] <= threshold and above_threshold and counter <= nFrames:
                    counter = counter + 1
        
                # Check if the value drops below the threshold again but buffer of nFrames is reached and the FP counter is bigger than 1
                elif row[col] <= threshold and above_threshold and counter >= nFrames and counterFP >= 2:
                    above_threshold = False
                    end_index = index  # Store the end index for this interval
        
                    # Find the highest number between crossing indices and sum them up
                    subset = currentCSV_transformed.loc[start_index:end_index, col]
                    highest_value = subset.max() if not subset.empty else 0  # Handling empty subset
                    sums_list.append(highest_value)
        
                    counter = 0
                    counterFP = 0
                    
                # Check if the value drops below the threshold again but buffer of nFrames is reached and the FP counter is not bigger than 1
                elif row[col] <= threshold and above_threshold and counter >= nFrames and counterFP <= 1:
                    above_threshold = False
                    counter = 0
                    counterFP = 0
        
                # Add the sums list to the sums dictionary
                sums_dict[col] = sums_list
        resultList = list(sums_dict.values())
        
        # Calculate the sum per element
        sum_per_element = [sum(sublist) for sublist in resultList if resultList]
        
        currentCSV['NHeuristic'] = sum_per_element 
        currentCSV = currentCSV[['videoname', 'NHeuristic']]

        outputName = "./04_Datasets/dataset_1/Nimproved_th_" + str(threshold) + "_nFrames_" + str(nFrames) + ".csv"
        currentCSV.to_csv(outputName)

/tmp/ipykernel_37463/2750438355.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sequences.fillna(0, inplace=True) #Padd with 0s
/tmp/ipykernel_37463/2750438355.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sequences.fillna(0, inplace=True) #Padd with 0s
/tmp/ipykernel_37463/2750438355.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sequences.fillna(0, inplace=True) #Padd with 0s
/tmp/ipykernel_37463/2750438355.py:21: S